In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pickle
import regex as re
import numpy as np
import pandas as pd
from datetime import datetime

In [ ]:
csv_file_path = '/content/drive/MyDrive/School/High-risk Project/Sepsis DataSet'

In [ ]:
!ls '/content/drive/MyDrive/School/High-risk Project/Sepsis DataSet'

ADMISSIONS.csv				  SEPSIS_POS_CHART_EVENTS_AGE_ABOVE_64.csv
clean_sepsis_neg.csv			  SEPSIS_POS_CHART_EVENTS.csv
clean_sepsis_pos.csv			  sepsis_positive_pivoted.csv
DIAGNOSES_ICD.csv			  SEPSIS_POS_LAB_EVENTS_AGE_ABOVE_64-1714062992998.csv
Sepsis_Analysis.sql			  SEPSIS_POS_LAB_EVENTS_AGE_ABOVE_64.csv
SEPSIS_NEG_CHART_EVENTS_AGE_ABOVE_64.csv  SEPSIS_POS_PAT_AGE.csv
SEPSIS_NEG_CHART_EVENTS.csv		  test_data.csv
SEPSIS_NEG_LAB_EVENTS_AGE_ABOVE_64.csv	  train_data.csv
SEPSIS_NEG_PAT_AGE.csv			  validate_data.csv
sepsis_neg_pivoted.csv


In [ ]:
###   LOAD DATA   ###
pat  = pd.read_csv(csv_file_path+'/SEPSIS_POS_PAT_AGE.csv')
charts  = pd.read_csv(csv_file_path+'/SEPSIS_POS_CHART_EVENTS_AGE_ABOVE_64.csv')
labs    = pd.read_csv(csv_file_path+'/SEPSIS_POS_LAB_EVENTS_AGE_ABOVE_64.csv')
diag = pd.read_csv(csv_file_path+'/DIAGNOSES_ICD.csv')

In [ ]:
adm = pd.read_csv(csv_file_path+'/ADMISSIONS.csv')

In [ ]:
adm.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1


In [ ]:
diag.ICD9_CODE = diag.ICD9_CODE.astype(str)
# Define the conditions to filter out rows
conditions = (diag['ICD9_CODE'].str.startswith('038') |
              diag['ICD9_CODE'].str.startswith('99591') |
              diag['ICD9_CODE'].str.startswith('99592') |
              diag['ICD9_CODE'].str.startswith('78552'))

# Filter rows that meet any of the conditions
diag_include = diag[conditions]
diag.info()
diag_include.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 651047 entries, 0 to 651046
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   ROW_ID      651047 non-null  int64  
 1   SUBJECT_ID  651047 non-null  int64  
 2   HADM_ID     651047 non-null  int64  
 3   SEQ_NUM     651000 non-null  float64
 4   ICD9_CODE   651047 non-null  object 
dtypes: float64(1), int64(3), object(1)
memory usage: 24.8+ MB
<class 'pandas.core.frame.DataFrame'>
Index: 14182 entries, 81 to 650934
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ROW_ID      14182 non-null  int64  
 1   SUBJECT_ID  14182 non-null  int64  
 2   HADM_ID     14182 non-null  int64  
 3   SEQ_NUM     14182 non-null  float64
 4   ICD9_CODE   14182 non-null  object 
dtypes: float64(1), int64(3), object(1)
memory usage: 664.8+ KB


In [ ]:
pat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1096 entries, 0 to 1095
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   subject_id  1096 non-null   int64
 1   age         1096 non-null   int64
dtypes: int64(2)
memory usage: 17.2 KB


In [ ]:
pat = pat[pat.age>64]

In [ ]:
pat = pat[pat.age<120]

In [ ]:
pat.shape

(566, 2)

In [ ]:
# make sure the patient is given a formal icd9_code diagnosis of sepsis during their stay
pat = pat[pat['subject_id'].isin(diag_include['SUBJECT_ID'])]
pat.shape

(473, 2)

In [ ]:
charts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312761 entries, 0 to 312760
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   subject_id    312761 non-null  int64  
 1   hadm_id       312761 non-null  int64  
 2   charttime     312761 non-null  object 
 3   lab_category  312761 non-null  object 
 4   lab_valuenum  312522 non-null  float64
 5   valueuom      312617 non-null  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 14.3+ MB


In [ ]:
labs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8203 entries, 0 to 8202
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   subject_id    8203 non-null   int64  
 1   hadm_id       8203 non-null   int64  
 2   charttime     8203 non-null   object 
 3   lab_category  8203 non-null   object 
 4   lab_valuenum  8190 non-null   float64
 5   valueuom      8203 non-null   object 
dtypes: float64(1), int64(2), object(3)
memory usage: 384.6+ KB


In [ ]:
charts_labs = pd.concat([charts,labs], ignore_index=True)

In [ ]:
charts_labs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320964 entries, 0 to 320963
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   subject_id    320964 non-null  int64  
 1   hadm_id       320964 non-null  int64  
 2   charttime     320964 non-null  object 
 3   lab_category  320964 non-null  object 
 4   lab_valuenum  320712 non-null  float64
 5   valueuom      320820 non-null  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 14.7+ MB


In [ ]:
sepsis_positive = pd.merge(pat, charts_labs, on=['subject_id'], how='inner')

In [ ]:
sepsis_positive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259742 entries, 0 to 259741
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   subject_id    259742 non-null  int64  
 1   age           259742 non-null  int64  
 2   hadm_id       259742 non-null  int64  
 3   charttime     259742 non-null  object 
 4   lab_category  259742 non-null  object 
 5   lab_valuenum  259535 non-null  float64
 6   valueuom      259627 non-null  object 
dtypes: float64(1), int64(3), object(3)
memory usage: 13.9+ MB


In [ ]:
sepsis_positive

,subject_id,age,hadm_id,charttime,lab_category,lab_valuenum,valueuom
0,21,88,111970,2135-01-30 22:00:00.000,Invasive Blood Pressure systolic,114.000000,mmHg
1,21,88,111970,2135-01-30 22:00:00.000,Heart Rate,79.000000,BPM
2,21,88,111970,2135-01-30 22:00:00.000,Temperature (Fahrenheit),99.699997,Deg. F
3,21,88,111970,2135-01-30 22:00:00.000,Invasive Blood Pressure mean,60.000000,mmHg
4,21,88,111970,2135-01-30 22:00:00.000,Respiratory Rate (RR),12.000000,BPM
...,...,...,...,...,...,...,...
259737,99299,76,166418,2161-03-10 22:00:00.000,Heart Rate,73.000000,bpm
259738,99299,76,166418,2161-03-09 21:20:00.000,White Blood Cells (cells/mm^3),10.600000,K/uL
259739,99299,76,166418,2161-03-10 04:58:00.000,White Blood Cells (cells/mm^3),6.300000,K/uL
259740,99299,76,166418,2161-03-11 15:15:00.000,White Blood Cells (cells/mm^3),4.300000,K/uL


In [ ]:
sepsis_positive["lab_value"] = round(sepsis_positive["lab_valuenum"],2).astype(str)

In [ ]:
sepsis_positive

,subject_id,age,hadm_id,charttime,lab_category,lab_valuenum,valueuom,lab_value
0,21,88,111970,2135-01-30 22:00:00.000,Invasive Blood Pressure systolic,114.000000,mmHg,114.0
1,21,88,111970,2135-01-30 22:00:00.000,Heart Rate,79.000000,BPM,79.0
2,21,88,111970,2135-01-30 22:00:00.000,Temperature (Fahrenheit),99.699997,Deg. F,99.7
3,21,88,111970,2135-01-30 22:00:00.000,Invasive Blood Pressure mean,60.000000,mmHg,60.0
4,21,88,111970,2135-01-30 22:00:00.000,Respiratory Rate (RR),12.000000,BPM,12.0
...,...,...,...,...,...,...,...,...
259737,99299,76,166418,2161-03-10 22:00:00.000,Heart Rate,73.000000,bpm,73.0
259738,99299,76,166418,2161-03-09 21:20:00.000,White Blood Cells (cells/mm^3),10.600000,K/uL,10.6
259739,99299,76,166418,2161-03-10 04:58:00.000,White Blood Cells (cells/mm^3),6.300000,K/uL,6.3
259740,99299,76,166418,2161-03-11 15:15:00.000,White Blood Cells (cells/mm^3),4.300000,K/uL,4.3


In [ ]:
sepsis_positive = sepsis_positive.drop(['lab_valuenum','valueuom'], axis=1)

In [ ]:
sepsis_positive

,subject_id,age,hadm_id,charttime,lab_category,lab_value
0,21,88,111970,2135-01-30 22:00:00.000,Invasive Blood Pressure systolic,114.0
1,21,88,111970,2135-01-30 22:00:00.000,Heart Rate,79.0
2,21,88,111970,2135-01-30 22:00:00.000,Temperature (Fahrenheit),99.7
3,21,88,111970,2135-01-30 22:00:00.000,Invasive Blood Pressure mean,60.0
4,21,88,111970,2135-01-30 22:00:00.000,Respiratory Rate (RR),12.0
...,...,...,...,...,...,...
259737,99299,76,166418,2161-03-10 22:00:00.000,Heart Rate,73.0
259738,99299,76,166418,2161-03-09 21:20:00.000,White Blood Cells (cells/mm^3),10.6
259739,99299,76,166418,2161-03-10 04:58:00.000,White Blood Cells (cells/mm^3),6.3
259740,99299,76,166418,2161-03-11 15:15:00.000,White Blood Cells (cells/mm^3),4.3


In [ ]:
sepsis_positive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259742 entries, 0 to 259741
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   subject_id    259742 non-null  int64 
 1   age           259742 non-null  int64 
 2   hadm_id       259742 non-null  int64 
 3   charttime     259742 non-null  object
 4   lab_category  259742 non-null  object
 5   lab_value     259742 non-null  object
dtypes: int64(3), object(3)
memory usage: 11.9+ MB


In [ ]:
sepsis_positive_bkp = sepsis_positive
# sepsis_positive = sepsis_positive_bkp

In [ ]:
sepsis_positive.charttime = pd.to_datetime(sepsis_positive.charttime, format = '%Y-%m-%d %H:%M:%S.%f', errors = 'coerce')

In [ ]:
sepsis_positive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259742 entries, 0 to 259741
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   subject_id    259742 non-null  int64         
 1   age           259742 non-null  int64         
 2   hadm_id       259742 non-null  int64         
 3   charttime     259742 non-null  datetime64[ns]
 4   lab_category  259742 non-null  object        
 5   lab_value     259742 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 11.9+ MB


In [ ]:
# Find the latest test results for each patient
sepsis_positive_start = sepsis_positive[['subject_id', 'lab_category', 'charttime']].groupby(['subject_id', 'lab_category']).min().reset_index()
sepsis_positive_start.columns = ['subject_id', 'lab_category', 'charttime']
sepsis_positive_start.head(20)

,subject_id,lab_category,charttime
0,21,Heart Rate,2135-01-30 22:00:00
1,21,Invasive Blood Pressure mean,2135-01-30 22:00:00
2,21,Invasive Blood Pressure systolic,2135-01-30 22:00:00
3,21,Non-Invasive Blood Pressure diastolic,2135-01-31 21:00:00
4,21,Respiratory Rate (RR),2135-01-30 22:00:00
5,21,Temperature (Fahrenheit),2135-01-30 22:00:00
6,21,White Blood Cells (cells/mm^3),2135-01-30 19:05:00
7,157,Heart Rate,2106-06-17 20:15:00
8,157,Invasive Blood Pressure mean,2106-06-17 20:15:00
9,157,Invasive Blood Pressure systolic,2106-06-17 20:15:00


In [ ]:
sepsis_positive_start['lab_category'].value_counts()

lab_category
Heart Rate                               449
Respiratory Rate (RR)                    449
White Blood Cells (cells/mm^3)           449
Temperature (Fahrenheit)                 270
Invasive Blood Pressure mean             266
Invasive Blood Pressure systolic         266
Non-Invasive Blood Pressure diastolic    156
Invasive Blood Pressure diastolic          7
Non-Invasive Blood Pressure systolic       7
Arterial Blood Pressure diastolic          3
Arterial Blood Pressure systolic           3
WBC Count (cells/uL)                       2
Name: count, dtype: int64

In [ ]:
# Define the list of lab categories to drop
categories_to_drop = [
    'Invasive Blood Pressure mean',
    'Invasive Blood Pressure systolic',
    'Non-Invasive Blood Pressure diastolic',
    'Invasive Blood Pressure diastolic',
    'Non-Invasive Blood Pressure systolic',
    'Arterial Blood Pressure diastolic',
    'Arterial Blood Pressure systolic',
    'WBC Count (cells/uL)'
]

# Drop rows with specified lab categories
sepsis_positive_start = sepsis_positive_start[~sepsis_positive_start['lab_category'].isin(categories_to_drop)]
sepsis_positive_start['lab_category'].value_counts()

lab_category
Heart Rate                        449
Respiratory Rate (RR)             449
White Blood Cells (cells/mm^3)    449
Temperature (Fahrenheit)          270
Name: count, dtype: int64

In [ ]:
# # Get subject_ids that have lab-category == 'Invasive Blood Pressure systolic'
# subject_ids_systolic = sepsis_positive_start[sepsis_positive_start['lab_category'] == 'Invasive Blood Pressure systolic']['subject_id'].unique()

# # Get subject_ids that have lab-category == 'Invasive Blood Pressure mean'
# subject_ids_mean = sepsis_positive_start[sepsis_positive_start['lab_category'] == 'Invasive Blood Pressure mean']['subject_id'].unique()

# # Get subject_ids that have 'Invasive Blood Pressure systolic' but not 'Invasive Blood Pressure mean'
# subject_ids_systolic_only = set(subject_ids_systolic) - set(subject_ids_mean)

# # Display the subject_ids that have 'Invasive Blood Pressure systolic' but not 'Invasive Blood Pressure mean'
# print(subject_ids_systolic_only)

In [ ]:
# Group by subject_id and find the min and max charttime values
subject_id_summary = sepsis_positive_start.groupby('subject_id')['charttime'].agg(['min', 'max'])

# Calculate the time difference between min and max charttime values
subject_id_summary['time_diff'] = subject_id_summary['max'] - subject_id_summary['min']

# Check if the time difference is less than 24 hours
subject_id_summary['within_24_hours'] = subject_id_summary['time_diff'] <= pd.Timedelta('1 day')

# Merge result with original DataFrame
sepsis_positive_start = pd.merge(sepsis_positive_start, subject_id_summary[['within_24_hours']], left_on='subject_id', right_index=True, how='left')


In [ ]:
sepsis_positive_start.head(30)

,subject_id,lab_category,charttime,within_24_hours
0,21,Heart Rate,2135-01-30 22:00:00,True
4,21,Respiratory Rate (RR),2135-01-30 22:00:00,True
5,21,Temperature (Fahrenheit),2135-01-30 22:00:00,True
6,21,White Blood Cells (cells/mm^3),2135-01-30 19:05:00,True
7,157,Heart Rate,2106-06-17 20:15:00,True
11,157,Respiratory Rate (RR),2106-06-17 20:15:00,True
12,157,Temperature (Fahrenheit),2106-06-17 20:15:00,True
13,157,White Blood Cells (cells/mm^3),2106-06-17 16:00:00,True
14,450,Heart Rate,2155-05-18 22:30:00,False
17,450,Respiratory Rate (RR),2155-05-18 22:30:00,False


In [ ]:
sepsis_positive_start['within_24_hours'].value_counts()

within_24_hours
True     1585
False      32
Name: count, dtype: int64

In [ ]:
# Filter rows where within_24_hours is True
sepsis_positive_start = sepsis_positive_start[sepsis_positive_start['within_24_hours'] == True]

In [ ]:
sepsis_positive_start['within_24_hours'].value_counts()

within_24_hours
True    1585
Name: count, dtype: int64

In [ ]:
sepsis_positive_start['lab_category'].value_counts()

lab_category
Heart Rate                        441
Respiratory Rate (RR)             441
White Blood Cells (cells/mm^3)    441
Temperature (Fahrenheit)          262
Name: count, dtype: int64

In [ ]:
df_sepsis = sepsis_positive.merge(sepsis_positive_start, how='inner', on=['subject_id', 'lab_category', 'charttime'])

In [ ]:
df_sepsis.shape

(1585, 7)

In [ ]:
df_sepsis[df_sepsis.subject_id == 94]

,subject_id,age,hadm_id,charttime,lab_category,lab_value,within_24_hours


In [ ]:
df_sepsis[df_sepsis.duplicated(['subject_id', 'lab_category'])]
# df_sepsis.groupby['subject_id','age','lab_category'].duplicated().sum()

,subject_id,age,hadm_id,charttime,lab_category,lab_value,within_24_hours


In [ ]:
df_sepsis[df_sepsis.subject_id == 21257]

,subject_id,age,hadm_id,charttime,lab_category,lab_value,within_24_hours


In [ ]:
# # 1714
# i = df_sepsis[((df_sepsis.subject_id == 21257) & ( df_sepsis.lab_category == 'Temperature (Fahrenheit)') & (df_sepsis.lab_value == float(32.0)))].index

In [ ]:
# df_sepsis.drop(index=1075, axis=0, inplace=True)

In [ ]:
df_sepsis[df_sepsis.duplicated(['subject_id', 'lab_category'])]

,subject_id,age,hadm_id,charttime,lab_category,lab_value,within_24_hours


In [ ]:
# Convert column names to lowercase
adm.columns = [column.lower() for column in adm.columns]

# Display the DataFrame with lowercase column names
adm.head()

,row_id,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1


In [ ]:
# Convert the column to datetime data type
adm['admittime'] = pd.to_datetime(adm['admittime'])

In [ ]:
# Merge the DataFrames on subject_id
merged_df = df_sepsis.merge(adm[['hadm_id','admittime']], on ='hadm_id')

In [ ]:
merged_df.head()

,subject_id,age,hadm_id,charttime,lab_category,lab_value,within_24_hours,admittime
0,21,88,111970,2135-01-30 22:00:00,Heart Rate,79.0,True,2135-01-30 20:50:00
1,21,88,111970,2135-01-30 22:00:00,Temperature (Fahrenheit),99.7,True,2135-01-30 20:50:00
2,21,88,111970,2135-01-30 22:00:00,Respiratory Rate (RR),12.0,True,2135-01-30 20:50:00
3,21,88,111970,2135-01-30 19:05:00,White Blood Cells (cells/mm^3),30.5,True,2135-01-30 20:50:00
4,157,81,107880,2106-06-17 20:15:00,Heart Rate,115.0,True,2106-06-17 19:51:00


In [ ]:
# Calculate the time difference between the two date columns
time_diff = merged_df['charttime'] - merged_df['admittime']

# Check if the time difference is less than or equal to 24 hours
within_24_hours = time_diff <= pd.Timedelta('1 day')

# Add the result as a new column in the DataFrame
merged_df['within_24_hours'] = within_24_hours

# Display the DataFrame
merged_df.head()

,subject_id,age,hadm_id,charttime,lab_category,lab_value,within_24_hours,admittime
0,21,88,111970,2135-01-30 22:00:00,Heart Rate,79.0,True,2135-01-30 20:50:00
1,21,88,111970,2135-01-30 22:00:00,Temperature (Fahrenheit),99.7,True,2135-01-30 20:50:00
2,21,88,111970,2135-01-30 22:00:00,Respiratory Rate (RR),12.0,True,2135-01-30 20:50:00
3,21,88,111970,2135-01-30 19:05:00,White Blood Cells (cells/mm^3),30.5,True,2135-01-30 20:50:00
4,157,81,107880,2106-06-17 20:15:00,Heart Rate,115.0,True,2106-06-17 19:51:00


In [ ]:
merged_df['within_24_hours'].value_counts()

within_24_hours
True    1585
Name: count, dtype: int64

In [ ]:
df = df_sepsis.pivot(index=['subject_id', 'age'], columns='lab_category', values='lab_value')
df.columns.name = None
df.reset_index(inplace=True)
df.head(50)

,subject_id,age,Heart Rate,Respiratory Rate (RR),Temperature (Fahrenheit),White Blood Cells (cells/mm^3)
0,21,88,79.0,12.0,99.7,30.5
1,157,81,115.0,30.0,98.4,52.0
2,507,75,104.0,30.0,95.6,11.4
3,545,85,70.0,25.0,96.0,2.4
4,576,88,99.0,26.0,96.2,46.2
5,588,69,93.0,11.0,98.4,14.8
6,618,78,102.0,23.0,97.6,21.5
7,690,79,92.0,17.0,100.1,17.3
8,711,84,114.0,18.0,95.0,15.6
9,781,76,81.0,18.0,96.8,10.8


In [ ]:
df.shape

(442, 6)

In [ ]:
# # List of columns to check for NaN values
# columns_to_check = ['Heart Rate', 'Respiratory Rate (RR)', 'Temperature (Fahrenheit)', 'White Blood Cells (cells/mm^3)']

# # Drop rows with NaN values in any of the specified columns
# df_cleaned = df.dropna(subset=columns_to_check, how='any', na_values=['NaN', 'nan'])
df.replace(['NaN', 'nan'], float('nan'), inplace=True)
df_cleaned = df.dropna()

# check shape of the cleaned DataFrame
df_cleaned.shape

(258, 6)

In [ ]:
df_cleaned

,subject_id,age,Heart Rate,Respiratory Rate (RR),Temperature (Fahrenheit),White Blood Cells (cells/mm^3)
0,21,88,79.0,12.0,99.7,30.5
1,157,81,115.0,30.0,98.4,52.0
2,507,75,104.0,30.0,95.6,11.4
3,545,85,70.0,25.0,96.0,2.4
4,576,88,99.0,26.0,96.2,46.2
...,...,...,...,...,...,...
290,31910,79,90.0,22.0,101.8,17.3
292,32059,82,110.0,19.0,97.4,22.2
293,32098,74,95.0,15.0,99.2,22.0
294,32124,86,120.0,15.0,98.2,11.3


In [ ]:
# Define a function to concatenate the unit strings to each value in the specified columns
def add_unit(column):
    if column.name == 'Heart Rate':
        return column.astype(str) + ' BPM'
    # elif column.name == 'Invasive Blood Pressure mean':
    #     return column.astype(str) + ' mmHg'
    elif column.name == 'Respiratory Rate (RR)':
        return column.astype(str) + ' BPM'
    elif column.name == 'Temperature (Fahrenheit)':
        return column.astype(str) + ' Deg. F'
    elif column.name == 'White Blood Cells (cells/mm^3)':
        return column.astype(str) + ' K/uL'
    else:
        return column

# Apply the function to the specified columns
df_updated = df_cleaned.apply(add_unit)

# Display the updated DataFrame
df_updated.head()

,subject_id,age,Heart Rate,Respiratory Rate (RR),Temperature (Fahrenheit),White Blood Cells (cells/mm^3)
0,21,88,79.0 BPM,12.0 BPM,99.7 Deg. F,30.5 K/uL
1,157,81,115.0 BPM,30.0 BPM,98.4 Deg. F,52.0 K/uL
2,507,75,104.0 BPM,30.0 BPM,95.6 Deg. F,11.4 K/uL
3,545,85,70.0 BPM,25.0 BPM,96.0 Deg. F,2.4 K/uL
4,576,88,99.0 BPM,26.0 BPM,96.2 Deg. F,46.2 K/uL


In [ ]:
# sepsis_positive["lab_test_value"] = sepsis_positive["lab_category"]+ " is " + sepsis_positive["lab_value"]
# grouped_df =
# sepsis_positive.groupby(['subject_id', 'age', 'lab_category'])['lab_value'].apply(list).reset_index()
# sepsis_positive.loc[sepsis_positive.groupby(['subject_id', 'age', 'lab_category']).charttime.idxmax()]
# sepsis_positive.sort_values('charttime').groupby(['subject_id', 'age', 'lab_category']).tail(1)

# sepsis_positive = sepsis_positive.drop(['lab_category','lab_value'], axis=1)
# out = grouped_df.pivot(index=['subject_id', 'age'], columns='lab_category', values='lab_value')
# print(out)

# sepsis_positive.sort_values(["subject_id","hadm_id", "charttime"], ignore_index=True, inplace=True)
# grouped_df = sepsis_positive.groupby(['subject_id', 'age', 'hadm_id', 'charttime'])['lab_test_value'].apply(list).reset_index()

# grouped_df.loc[grouped_df.subject_id == 99973, 'lab_test_value']
# print(grouped_df[['lab_test_value']].head(10))
# grouped_df[grouped_df.subject_id == 99973]

In [ ]:
# Drop the column
df_updated.drop(columns=['age', 'subject_id'], inplace=True)

# Display the DataFrame after dropping the column
df_updated.head()

,Heart Rate,Respiratory Rate (RR),Temperature (Fahrenheit),White Blood Cells (cells/mm^3)
0,79.0 BPM,12.0 BPM,99.7 Deg. F,30.5 K/uL
1,115.0 BPM,30.0 BPM,98.4 Deg. F,52.0 K/uL
2,104.0 BPM,30.0 BPM,95.6 Deg. F,11.4 K/uL
3,70.0 BPM,25.0 BPM,96.0 Deg. F,2.4 K/uL
4,99.0 BPM,26.0 BPM,96.2 Deg. F,46.2 K/uL


In [ ]:
df_updated.to_csv(csv_file_path+'/clean_sepsis_pos.csv')

In [ ]:
!ls '/content/drive/MyDrive/School/High-risk Project/Sepsis DataSet'

ADMISSIONS.csv				  sepsis_neg_pivoted.csv
clean_sepsis_pos.csv			  SEPSIS_POS_CHART_EVENTS_AGE_ABOVE_64.csv
DIAGNOSES_ICD.csv			  SEPSIS_POS_CHART_EVENTS.csv
Sepsis_Analysis.sql			  sepsis_positive_pivoted.csv
SEPSIS_NEG_CHART_EVENTS_AGE_ABOVE_64.csv  SEPSIS_POS_LAB_EVENTS_AGE_ABOVE_64-1714062992998.csv
SEPSIS_NEG_CHART_EVENTS.csv		  SEPSIS_POS_LAB_EVENTS_AGE_ABOVE_64.csv
SEPSIS_NEG_LAB_EVENTS_AGE_ABOVE_64.csv	  SEPSIS_POS_PAT_AGE.csv
SEPSIS_NEG_PAT_AGE.csv
